In [1]:
!pip install biosppy
!pip install antropy

     |████████████████████████████████| 85 kB 2.3 MB/s 
  Created wheel for biosppy: filename=biosppy-0.7.3-py2.py3-none-any.whl size=95430 sha256=ea2cc4e1938632d0cb5cb7bce7e8412bd5c3d379a03c5147c05f13505fcc5901
  Stored in directory: /root/.cache/pip/wheels/2f/4f/8f/28b2adc462d7e37245507324f4817ce1c64ef2464f099f4f0b
Successfully built biosppy
     |████████████████████████████████| 49 kB 3.7 MB/s 
  Created wheel for antropy: filename=antropy-0.1.4-py3-none-any.whl size=16158 sha256=6b5c80ccc349d2b7393d390f8ddada00f3aa4e0c329197afcee9c9f14712ca47
  Stored in directory: /root/.cache/pip/wheels/d4/cc/6b/28fa7036bdabc6f1f92fd6a108ae259dbb48f107f676d6b491
Successfully built antropy


In [2]:
import biosppy.signals.ecg as ecg
import antropy as ant
import matplotlib.pyplot as plt

In [3]:
import torch
import numpy as np
import pandas as pd

train_X = pd.read_csv('X_train.csv')
train_X_tensor = torch.tensor(train_X.values)[:,1:]
train_y = pd.read_csv("y_train.csv")
train_y_tensor = torch.tensor(train_y.values)[:,1:]

test_X = pd.read_csv('X_test.csv')
test_X_tensor = torch.tensor(test_X.values)[:,1:]

In [4]:
test_X_tensor.shape

torch.Size([3411, 17842])

In [5]:
X_train = train_X_tensor.numpy()
y_train = train_y_tensor.numpy()

In [6]:
y_train.shape

(5117, 1)

In [7]:
X_train.shape

(5117, 17842)

In [8]:
y_train = np.squeeze(y_train, axis=1)

In [ ]:
#@title hide pls
signal = X_train[1]
signal = signal[~np.isnan(signal)] #drop nans
r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
# print(r_peaks)

if len(r_peaks) >= 2:
    #print(ecg.extract_heartbeats(signal, r_peaks, 300))
    beats = ecg.extract_heartbeats(signal, r_peaks, 300)['templates']
beats.shape
r_peaks

In [ ]:
!pip install pyhrv
from pyhrv.frequency_domain import welch_psd

In [10]:
fbands = {'ulf': (0.0, 0.033), 'vlf': (0.033, 0.04), 'lf': (0.04, 0.15), 'hf': (0.15, 0.4)} #WHY CANT I ADD 5th frequency?
# fbands = {'vlf': (0.033, 0.04), 'lf': (0.04, 0.15), 'hf': (0.15, 0.4),"noise":(0.4,100)} #WHY CANT I ADD 5th frequency?

In [ ]:
#@title hide pls
signal = X_train[0]
signal = signal[~np.isnan(signal)] #drop nans

res = ecg.ecg(signal, sampling_rate=300, show=False)
r_peaks = res['rpeaks']
heart_rates = res['heart_rate']

# else:
#   print(f"len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
# r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
# extracted_rpeaks.append([signal,r_peaks])
print(heart_rates)
print(r_peaks.shape)

In [11]:
extracted_rpeaks = []

In [13]:
max = 0
for i in range(len(X_train)):
    signal = X_train[i]
    signal = signal[~np.isnan(signal)] #drop nans
    if (i%500==0):
      print(i)
    res = ecg.ecg(signal, sampling_rate=300, show=False)
    r_peaks = res['rpeaks']
    heart_rates = res["heart_rate"]
    if len(r_peaks)>max:
      max = len(r_peaks)
    if len(r_peaks) < 5:
      print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # else:
    #   print(f"len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
    extracted_rpeaks.append([signal,r_peaks,heart_rates])
print(max)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
160


In [ ]:
min = 150
for i in range(5117):
  r_peaks = extracted_rpeaks[i][1]
  if len(r_peaks)<min:
    min = len(r_peaks)
min

In [14]:
len(extracted_rpeaks)

5117

In [ ]:
#@title pls hide yeo
count =0
diffs = []
for i in range(len(y_train[y_train==3])):
  signal = X_train[y_train==3][i]
  label = y_train[y_train==3][i]
  signal = signal[~np.isnan(signal)] #drop nans

  res = ecg.ecg(signal, sampling_rate=300, show=False, )
  r_peaks = res['rpeaks']
  heart_rates = res["heart_rate"]
  beats = res["templates"]
  filtered = res["filtered"]
  r_peaks_s = r_peaks/300
  freqs = 20*np.log10(np.abs(np.fft.rfft(signal)))
  f = np.linspace(0, 300/2, len(freqs))
  plt.plot(f, freqs)
  # a = welch_psd(rpeaks=r_peaks_s, fbands = fbands, show=False, show_param=False, legend=False)#, fbands=None)
  # res_filt = ecg.ecg(filtered, sampling_rate=300, show=False, )
  # beats_filt = res_filt["templates"]
  # print(np.mean(np.mean(beats,axis=0)-np.mean(beats_filt,axis=0)))

  #print(f"num of rpeaks if {len(r_peaks)} and avg heartrate is {np.mean(heart_rates)}")
  #print(f"I have {len(r_peaks)} pulses but I should have {np.mean(heart_rates[2:])*len(signal)/18000}")
  # c = np.mean(sorted(heart_rates[:-2]))*len(signal)/18000
  # diff = np.abs((len(r_peaks)/c)-1)
  # print(diff)
  # diffs.append(diff)
# print(np.mean(np.array(diffs)))

  # print(r_peaks)
  # print(r_peaks.shape)
  # print(heart_rates[:15])
  # print(heart_rates.shape)
  # print(beats)
  # print(beats.shape)

In [ ]:
import scipy.signal

In [ ]:
signal = X_train[y_train==0][8]
label = y_train[y_train==3][0]
signal = signal[~np.isnan(signal)] #drop nans
f, Pxx_spec = scipy.signal.welch(signal, 300, 'flattop', 1024, scaling='spectrum')
Pxx_spec.shape

(513,)

In [ ]:
Pxx_spec[0::10].shape

(52,)

In [ ]:
#@title hide yo
# signal = X_train[y_train==0][6]
# label = y_train[y_train==3][0]
# signal = signal[~np.isnan(signal)] #drop nans
# res = ecg.ecg(signal, sampling_rate=300, show=False, )
# r_peaks = res['rpeaks']
# heart_rates = res["heart_rate"]
# beats = res["templates"]
# filtered = res["filtered"]
# r_peaks_s = r_peaks/300
# freqs = 20*np.log10(np.abs(np.fft.rfft(signal)))
# f = np.linspace(0, 300/2, len(freqs))
# len(freqs)

4122

In [18]:
import scipy as scipy

In [ ]:
np.histogram([1,2,3,4,5],bins=2)[1]

array([1., 3., 5.])

In [40]:
extracted_features_l = np.ones((5117,5*27+5*4+2+2+2+2*4+1+1+61+10+45))

In [ ]:
for i in range(len(extracted_rpeaks)):
    signal, r_peaks, heart_rates = (extracted_rpeaks[i][0],extracted_rpeaks[i][1],extracted_rpeaks[i][2])
    if (i%500==0):
      print(f"processing point {i}")
    if len(r_peaks) >= 5:
        #print(ecg.extract_heartbeats(signal, r_peaks, 300))
        beats = ecg.extract_heartbeats(signal, r_peaks, 300,before=0.45,after=0.45)['templates']
        if len(beats) != 0:
            r_diffs = np.diff(r_peaks)
            mu = np.mean(beats, axis=0)[0::10] 
            mini = np.amin(beats, axis=0)[0::10] 
            maxa = np.amax(beats, axis=0)[0::10]  
            var = np.std(beats, axis=0)[0::10] 
            med = np.median(beats, axis=0)[0::10]
            r_mu = np.mean(r_diffs)
            r_mini = np.amin(r_diffs) 
            r_maxa = np.amax(r_diffs)  
            r_var = np.std(r_diffs)
            r_med = np.median(r_diffs)
            r_kurtosis= scipy.stats.kurtosis(r_diffs)
            r_skew = scipy.stats.skew(r_diffs)
            r_20 = np.percentile(r_diffs,20)
            r_40 = np.percentile(r_diffs,40)
            r_60 = np.percentile(r_diffs,60)
            r_80 = np.percentile(r_diffs,80)
            r_hist = np.histogram(r_diffs,bins=14)[1]
            smentropy = ant.sample_entropy(mu)
            appentropy = ant.app_entropy(mu)

            r_diffs_diffs = np.diff(r_diffs)
            r_diffs_mu = np.mean(r_diffs_diffs)
            r_diffs_maxa = np.amax(r_diffs_diffs)
            r_diffs_mini = np.amin(r_diffs_diffs)
            r_diffs_var = np.std(r_diffs_diffs)
            r_diffs_med = np.median(r_diffs_diffs)
            kurtosis= scipy.stats.kurtosis(signal)
            skew = scipy.stats.skew(signal)
            r_diffs_25 = np.percentile(r_diffs_diffs,25)
            r_diffs_75 = np.percentile(r_diffs_diffs,75)
            nn_50 = len(r_diffs_diffs[r_diffs_diffs>15]) / len(r_diffs_diffs)
            nn_20 = len(r_diffs_diffs[r_diffs_diffs>6]) / len(r_diffs_diffs)
            c = np.mean(sorted(heart_rates[:-2]))*len(signal)/18000
            rat = np.abs((len(r_peaks)/c)-1)
            cv_rr = r_var/r_mu
            cv_drr = r_diffs_var/r_mu
            r_diffs_hist = np.histogram(r_diffs_diffs,bins=14)[1]

            r_diffs_diffs_2 = np.diff(r_diffs_diffs)
            r_diffs2_mu = np.mean(r_diffs_diffs_2)
            r_diffs2_maxa = np.amax(r_diffs_diffs_2)
            r_diffs2_mini = np.amin(r_diffs_diffs_2)
            r_diffs2_var = np.std(r_diffs_diffs_2)
            r_diffs2_med = np.median(r_diffs_diffs_2)
            r_diffs2 = np.array([r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med])
            r_diffs_2_hist = np.histogram(r_diffs_diffs_2,bins=14)[1]

            abla, lospowers = scipy.signal.welch(signal, 300, 'flattop', 1024, scaling='spectrum')
            #heart rate stats
            if len(heart_rates)>0:
              hr_mu = np.mean(heart_rates)
              hr_maxa = np.amax(heart_rates)
              hr_mini = np.amin(heart_rates)
              hr_var = np.std(heart_rates)
              hr_med = np.median(heart_rates)
            else:
              hr_mu = np.nan
              hr_maxa = np.nan
              hr_mini = np.nan
              hr_var = np.nan
              hr_med = np.nan
            r_stats = np.array([r_mu,r_var,r_mini,r_maxa,r_med,r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med, r_diffs_25, r_diffs_75,hr_mu,hr_maxa,hr_mini,hr_var,hr_med,nn_50,nn_20,smentropy,appentropy,rat,cv_rr,cv_drr,r_skew,r_kurtosis,skew,kurtosis,r_20,r_40,r_60,r_80])
            r_stats = np.concatenate((r_stats,r_diffs2,lospowers[0:10],lospowers[10::10],r_hist,r_diffs_hist,r_diffs_2_hist))
            #HRV STATS

            r_peaks_s = r_peaks/300
            a = welch_psd(rpeaks=r_peaks_s, fbands = fbands, show=False, show_param=False, legend=False)#, fbands=None)
            hrv_stats = np.concatenate((a["fft_peak"],a["fft_abs"],np.array([a["fft_ratio"]])))

            extracted_features_l[i] = np.array(np.concatenate((mu,var,mini,maxa,med,r_stats,hrv_stats)))
    else:  
        print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
        extracted_features_l[i]=np.ones(5*27+5*4+2+2+2+2*4+1+1+61+10+45) #np.array([np.ones(18),np.ones(18),np.ones(18),np.ones(18)])

In [ ]:
#@title My title
# for i in range(len(extracted_rpeaks)):

#     signal = signal[~np.isnan(signal)] #drop nans
#     r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
#     # print(r_peaks)

#     if len(r_peaks) >= 5:
#         #print(ecg.extract_heartbeats(signal, r_peaks, 300))
#         beats = ecg.extract_heartbeats(signal, r_peaks, 300)['templates']
#         if len(beats) != 0:
#             r_diffs = np.diff(r_peaks)
#             mu = np.mean(beats, axis=0)[0::10] 
#             mini = np.amin(beats, axis=0)[0::10] 
#             maxa = np.amax(beats, axis=0)[0::10]  
#             var = np.std(beats, axis=0)[0::10] 
#             r_mean = np.mean(r_diffs)
#             r_mini = np.amin(r_diffs) 
#             r_maxa = np.amax(r_diffs)  
#             r_var = np.std(r_peaks)
#             r_med = np.median(r_peaks)
#             # md = np.median(beats, axis=0)
#             r_stats = np.array([r_mean,r_mini,r_maxa,r_var,r_med])

#             #HRV STATS
#             r_peaks_s = r_peaks/300
#             a = welch_psd(rpeaks=r_peaks_s, show=False, show_param=False, legend=False)#, fbands=None)
#             hrv_stats = np.concatenate((a["fft_peak"],a["fft_rel"],a["fft_abs"],a["fft_log"]))
#             return np.array(np.concatenate((mu,var,mini,maxa,r_stats,hrv_stats)))
#         print(f"Inappropriate input encountered: len(r_peaks) was >=5 but len(beats)=0 => peaks = {len(r_peaks)}, len(beats)={len(beats)}, signal length ={len(signal)}")
#     print(f"Inappropriate input encountered: len(r_peaks) was <5 => peaks = {len(r_peaks)}, signal length ={len(signal)}")
#     return np.ones(4*18+5+4*3) #np.array([np.ones(18),np.ones(18),np.ones(18),np.ones(18)])

In [ ]:
for i in range(len(extracted_rpeaks)):
    signal, r_peaks, heart_rates = (extracted_rpeaks[i][0],extracted_rpeaks[i][1],extracted_rpeaks[i][2])
    if (i%500==0):
      print(f"processing point {i}")
    signal

In [ ]:
# extracted_features = np.apply_along_axis(extract_features, 1, X_train)

In [48]:
bool_extracted = (extracted_features_l == np.array(np.ones(5*27+5*4+2+2+2+2*4+1+1+61+10+45))).all(axis=1)

In [49]:
bool_extracted

array([False, False, False, ..., False, False, False])

In [50]:
extracted_features_valid = extracted_features_l[~bool_extracted]

In [51]:
extracted_features_valid.shape

(5117, 287)

In [52]:
extracted_features_collapsed = extracted_features_valid#np.concatenate(np.stack(extracted_features_valid,axis=1),axis=1)
labels_collapsed = y_train[~bool_extracted]

In [53]:
extracted_features_collapsed.shape

(5117, 287)

In [ ]:
# #check if all inappropriate ones are noise:
# hopefully_noise_3 = y_train[bool_extracted]
# hopefully_noise_3

------------------------------------------------------Classification------------------------------------------------

In [55]:
#Training-val split
extracted_features_collapsed_a = extracted_features_collapsed[:4660]
labels_collapsed_a = labels_collapsed[:4660]
extracted_features_collapsed_b = extracted_features_collapsed[4660:]
labels_collapsed_b = labels_collapsed[4660:]

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_sample_weight

In [56]:
#Weigh examples
weighted_labels = compute_sample_weight( class_weight= "balanced" , y =labels_collapsed)
weighted_labels 

array([0.42219472, 0.42219472, 0.42219472, ..., 0.42219472, 0.42219472,
       0.86787653])

In [ ]:
#@title hide pls
# !pip install pytorch-tabnet
# from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
# tab = TabNetClassifier(n_d=24,n_a=24)  #TabNetRegressor()
# tab.fit(extracted_features_collapsed_a[:4300], labels_collapsed_a[:4300],eval_set=[(extracted_features_collapsed_a[4300:], labels_collapsed_a[4300:])],patience=150, max_epochs=1000)
# np.sum(labels_collapsed_b == tab.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)
# #got 0.723, and it even didnt train on the last 300 datapoints

In [ ]:
# weighted_labels[labels_collapsed==0] *=0.8
# weighted_labels[labels_collapsed==1] *=1.2
# weighted_labels[labels_collapsed==2] *=1.2
# weighted_labels[labels_collapsed==3] *=1.5
# weighted_labels[labels_collapsed==0] *=0.7
# weighted_labels[labels_collapsed==1] *=1.4
# weighted_labels[labels_collapsed==2] *=1.4
# weighted_labels[labels_collapsed==3] *=1.7

In [32]:
!pip install lightgbm

In [33]:
from lightgbm import LGBMClassifier

In [57]:
lgb = LGBMClassifier(n_estimators=1500, min_child_weight=3, num_leaves=20 ,learning_rate=0.1, max_depth=10, random_state=0, reg_lambda=5)
lgb.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4660])
np.sum(labels_collapsed_b == lgb.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)
#851

0.8643326039387309

In [235]:
importances = lgb.feature_importances_

In [255]:
extracted_pruned =  extracted_features_collapsed[:,importances>sorted(importances)[10]]
extracted_pruned_a = extracted_pruned[:4660]
extracted_pruned_b =  extracted_pruned[4660:]

In [254]:
sorted(importances)[10]

29

In [257]:
lgb_pru = LGBMClassifier(n_estimators=1500, min_child_weight=3, learning_rate=0.1, max_depth=10, random_state=0)
lgb_pru.fit(extracted_pruned_a, labels_collapsed_a, weighted_labels[:4660])
np.sum(labels_collapsed_b == lgb_pru.predict(extracted_pruned_b))/len(labels_collapsed_b)

0.8358862144420132

In [ ]:
a = lgb3.predict(extracted_features_collapsed_b)==lgb.predict(extracted_features_collapsed_b)
len(a[a==True])

385

In [35]:
#@title best settings so far
# lgb = LGBMClassifier(n_estimators=1500, min_child_weight=3, learning_rate=0.1, max_depth=10, random_state=0, reg_lambda=5)
# lgb.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4660])
# np.sum(labels_collapsed_b == lgb.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)
#851

In [ ]:
pred = lgb.predict(extracted_features_collapsed_b)
tot = 0
count0=0
count1=0
count2=0
count3=0

for i in range(5117-4660):
  signal, r_peaks, heart_rates = (extracted_rpeaks[i+4660][0],extracted_rpeaks[i+4660][1],extracted_rpeaks[i+4660][2])
  if pred[i]!=labels_collapsed_b[i]:
    print(f"actual label is {labels_collapsed_b[i]} predicted is {pred[i]}")
    if labels_collapsed_b[i]==0:
      count0+=1
    if labels_collapsed_b[i]==1:
      count1+=1
    if labels_collapsed_b[i]==2:
      count2+=1
    if labels_collapsed_b[i]==3:
      count3+=1
# i=0
# if labels_collapsed_b[i]==0:
# signal, r_peaks, heart_rates = (extracted_rpeaks[i+4660][0],extracted_rpeaks[i+4660][1],extracted_rpeaks[i+4660][2])
# if pred[i]!=labels_collapsed_b[i]:
#   print(f"actual label is {labels_collapsed_b[i]} predicted is {pred[i]}")
#   plt.plot

In [64]:
# lgb = LGBMClassifier(n_estimators=2000, min_child_weight=3, learning_rate=0.1, max_depth=10, random_state=0, reg_lambda=0.5)
# lgb.fit(extracted_features_collapsed, labels_collapsed, weighted_labels)
# np.sum(labels_collapsed == lgb.predict(extracted_features_collapsed))/len(labels_collapsed)
lgb = LGBMClassifier(n_estimators=1500, min_child_weight=3, num_leaves=20 ,learning_rate=0.1, max_depth=10, random_state=0, reg_lambda=5)
lgb.fit(extracted_features_collapsed, labels_collapsed, weighted_labels)
np.sum(labels_collapsed == lgb.predict(extracted_features_collapsed))/len(labels_collapsed)
#851

1.0

In [ ]:
!pip install xgboost
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators=1250, min_child_weight=3, learning_rate=0.145, max_depth=9, random_state=0)
xgb.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4660])
np.sum(labels_collapsed_b == xgb.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.838074398249453

In [ ]:
xgb = XGBClassifier(n_estimators=1250, min_child_weight=3, learning_rate=0.1, max_depth=9, random_state=0)
xgb.fit(extracted_features_collapsed, labels_collapsed, weighted_labels)
np.sum(labels_collapsed == xgb.predict(extracted_features_collapsed))/len(labels_collapsed)

In [ ]:
np.sum(labels_collapsed_a == xgb.predict(extracted_features_collapsed_a))/len(labels_collapsed_a)

1.0

----------------------------------NN APPROACH---------------------------

In [201]:
from sklearn.neural_network import MLPClassifier

In [209]:
extracted_features_collapsed -> 147,148,149,150,151,154,156,286

True

In [315]:
#remove nan columns
extracted_features_nonan = np.concatenate((extracted_features_collapsed[:,:147],extracted_features_collapsed[:,152:154],extracted_features_collapsed[:,155][:,np.newaxis],extracted_features_collapsed[:,157:286]),axis=1)

In [316]:
np.isnan(extracted_features_nonan).any()

False

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
mlp = make_pipeline(StandardScaler(),MLPClassifier(hidden_layer_sizes=(256, 128, 64), max_iter=1000, alpha=2, verbose=True, solver="adam", learning_rate_init= 0.0005, learning_rate="adaptive", batch_size=512, n_iter_no_change=300))
mlp.fit(extracted_features_nonan[:4660], labels_collapsed_a)
np.sum(labels_collapsed_b == mlp.predict(extracted_features_nonan[4660:]))/len(labels_collapsed_b)
#825

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
svc = make_pipeline(StandardScaler(), SVC(gamma='auto', C=5.5))
svc.fit(extracted_features_nonan[:4660], labels_collapsed_a)
np.sum(labels_collapsed_b == svc.predict(extracted_features_nonan[4660:]))/len(labels_collapsed_b)
#8.27

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def twoLayerFeedForward():
    clf = Sequential()
    clf.add(Dense(9, activation='relu', input_dim=3))
    clf.add(Dense(9, activation='relu'))
    clf.add(Dense(3, activation='softmax'))
    clf.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=["accuracy"])
    return clf

----------------------------SKORCH NNs AHEAD-------------------------

In [338]:
import torch as torch
from torch.nn import Linear, Dropout, Module
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler

In [395]:
class NeuralNet(Module):
    def __init__(self, num_units=10, dropout=0.1):
        super(NeuralNet, self).__init__()
        self.num_units = num_units
        self.linear_1 = Linear(279, 512)
        self.dropout = Dropout(0.2)
        self.linear_2 = Linear(512, 256)
        self.dropout = Dropout(0.2)
        self.linear_3 = Linear(256, 128)
        
        self.dropout = Dropout(0.1)
        self.linear_4 = Linear(128, 64)
        self.dropout = Dropout(0.1)
        self.linear_5 = Linear(64, 4)


    def forward(self, x):

        x = self.linear_1(x)
        x = F.relu(x)
        x = self.linear_2(x)
        x = F.relu(x)
        x = self.linear_3(x)

        x = F.relu(x)
        x = self.linear_4(x)
        x = F.relu(x)
        x = self.linear_5(x)
        x = F.softmax(x, dim=-1)

        return x

In [ ]:
!pip install skorch

In [331]:
# Import SKORCH NN classifier
from skorch import NeuralNetClassifier
from torch import optim

In [404]:
X_train[:,0::15].shape

(5117, 1190)

In [400]:
# The Neural Net is initialized with fixed hyperparameters
nn = make_pipeline(StandardScaler(), NeuralNetClassifier(NeuralNet, max_epochs=250, lr=0.0001, optimizer__weight_decay=0.1, batch_size=512, optimizer=optim.Adam))
# Training
# nn.fit(self.x, self.y)

In [401]:
nn.fit(extracted_features_nonan[:4660].astype(np.float32), labels_collapsed_a.astype(np.int_))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.3844       0.2876        1.3813  0.3855
      2        1.3794       0.2876        1.3764  0.3901
      3        1.3747       0.2876        1.3717  0.4098
      4        1.3701       0.2876        1.3671  0.3840
      5        1.3655       0.2886        1.3623  0.3949
      6        1.3608       0.2897        1.3574  0.3917
      7        1.3558       0.2918        1.3522  0.3788
      8        1.3505       0.2940        1.3465  0.3884
      9        1.3448       0.2972        1.3404  0.3941
     10        1.3385       0.3004        1.3336  0.3880
     11        1.3315       0.3036        1.3259  0.4058
     12        1.3237       0.3734        1.3173  0.3875
     13        1.3147       0.5655        1.3073  0.3849
     14        1.3044       0.6427        1.2956  0.3899
     15        1.2923       0.6513        1.2819  0.3891
     16        1.2779       0.6

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('neuralnetclassifier',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=NeuralNet(
    (linear_1): Linear(in_features=279, out_features=512, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear_2): Linear(in_features=512, out_features=256, bias=True)
    (linear_3): Linear(in_features=256, out_features=128, bias=True)
    (linear_4): Linear(in_features=128, out_features=64, bias=True)
    (linear_5): Linear(in_features=64, out_features=4, bias=True)
  ),
))])

In [374]:
np.isnan(extracted_features_nonan).any()

False

In [399]:
np.sum(labels_collapsed_b == nn.predict(extracted_features_nonan[4660:].astype(np.float32)))/len(labels_collapsed_b)

0.7811816192560175

----------------------------------------OTHER KINDA POINTLESS STUFF---------------------------

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_units=10, dropout=0.1):
        super(NeuralNet, self).__init__()
        self.num_units = num_units
        self.linear_1 = nn.Linear(13, num_units)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(num_units, 10)
        self.linear_3 = nn.Linear(10, 3)

    def forward(self, x):

        x = self.linear_1(x)
        x = F.relu(x)
        x = self.linear_2(x)
        x = F.relu(x)
        x = self.linear_3(x)
        x = F.softmax(x, dim=-1)

        return x

In [195]:
X_train_nn = X_train[:,0::15]

In [196]:
X_train_nn.shape

(5117, 1190)

In [197]:
batch_size = 1024
train_data = TensorDataset(torch.from_numpy(X_train_nn), torch.from_numpy(y_train))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [200]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [ ]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.clock()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.clock()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [ ]:
for x in range(len(X_train)):

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['landmarks'].size())

2-step process----------------------------------------

In [ ]:
new_dps = extracted_features_collapsed
new_labs = np.ones(5117)

In [ ]:
new_labs[~(labels_collapsed==3)] = 0
new_labs[labels_collapsed==3] = 1

In [ ]:
new_dps_a = new_dps[:4660]
new_labs_a = new_labs[:4660]
new_dps_b = new_dps[4660:]
new_labs_b = new_labs[4660:]

In [ ]:
weighted_labels_noise = compute_sample_weight(class_weight= "balanced" , y =new_labs)
np.unique(weighted_labels_noise)

array([ 0.51718213, 15.05      ])

In [ ]:
lgb_denoise = LGBMClassifier(n_estimators=1500, learning_rate=0.1, min_child_weight=3, max_depth=9, random_state=0)
lgb_denoise.fit(new_dps_a, new_labs_a, weighted_labels_noise[:4660])
np.sum(new_labs_b== lgb_denoise.predict(new_dps_b))/len(new_labs_b)

0.9693654266958425

In [ ]:
new_dps_nono = extracted_features_collapsed[labels_collapsed!=3]
new_labs_nono = labels_collapsed[labels_collapsed!=3]

In [ ]:
new_dps_nono_a = new_dps_nono[:4500]
new_dps_nono_b = new_dps_nono[4500:]
new_labs_nono_a = new_labs_nono[:4500]
new_labs_nono_b = new_labs_nono[4500:]

In [ ]:
weighted_labels_nono = compute_sample_weight(class_weight= "balanced" , y =new_labs_nono)
np.unique(weighted_labels_nono)

array([0.54422442, 1.11872456, 3.72234763])

In [ ]:
lgb_proper = LGBMClassifier(n_estimators=1500, learning_rate=0.1, min_child_weight=3, max_depth=9, random_state=0)
lgb_proper.fit(new_dps_nono_a, new_labs_nono_a, weighted_labels_nono[:4500])
np.sum(new_labs_nono_b== lgb_proper.predict(new_dps_nono_b))/len(new_labs_nono_b)

0.8523489932885906

-----------------------------------OTHER METHODS---------------------------------

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
mlp = make_pipeline(StandardScaler(),MLPClassifier(hidden_layer_sizes=(256, 128, 64), max_iter=1000, alpha=2, verbose=True, solver="adam", learning_rate_init= 0.0005, learning_rate="adaptive", batch_size=512, n_iter_no_change=300))
mlp.fit(extracted_features_collapsed_a[:,:-1], labels_collapsed_a)
np.sum(labels_collapsed_b == mlp.predict(extracted_features_collapsed_b[:,:-1]))/len(labels_collapsed_b)
#gives 0.781

In [ ]:
mlp = make_pipeline(StandardScaler(),MLPClassifier(hidden_layer_sizes=(512, 256, 128), max_iter=1000, alpha=1, verbose=True, solver="adam", learning_rate_init= 0.0005, learning_rate="adaptive", batch_size=512, n_iter_no_change=300))
mlp.fit(extracted_features_collapsed_e, labels_collapsed)
np.sum(labels_collapsed == mlp.predict(extracted_features_collapsed_e))/len(labels_collapsed)
#gives 0.738 for alpha=1, max_iters=4000

In [ ]:
np.sum(labels_collapsed_a == mlp.predict(extracted_features_collapsed_a))/len(labels_collapsed_a)

1.0

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = make_pipeline(StandardScaler(),KNeighborsClassifier(n_neighbors=25, weights='uniform', p=2))
neigh.fit(extracted_features_collapsed_a, labels_collapsed_a)
np.sum(labels_collapsed_b == neigh.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

In [ ]:
np.isnan(extracted_features_collapsed[:,153:]).any()

True

In [ ]:
extracted_features_collapsed_a_e = np.concatenate((extracted_features_collapsed_a[:,:-9],extracted_features_collapsed_a[:,-6:-3]),axis=1)

In [ ]:
extracted_features_collapsed_b_e = np.concatenate((extracted_features_collapsed_b[:,:-9],extracted_features_collapsed_b[:,-6:-3]), axis=1)

In [ ]:
extracted_features_collapsed_e = np.concatenate((extracted_features_collapsed[:,:154],extracted_features_collapsed[:,155:]), axis=1)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
svc = make_pipeline(StandardScaler(), SVC(gamma='auto', C=6.5))
svc.fit(extracted_features_collapsed_e[:4660], labels_collapsed_a)
np.sum(labels_collapsed_b == svc.predict(extracted_features_collapsed_e[:4660]))/len(labels_collapsed_b)

ValueError: ignored

In [ ]:
from sklearn.ensemble import BaggingClassifier
bag = make_pipeline(StandardScaler(), BaggingClassifier(n_estimators=1000, random_state=0))
bag.fit(extracted_features_collapsed_a[:,:-1], labels_collapsed_a)
np.sum(labels_collapsed_b == bag.predict(extracted_features_collapsed_b[:,:-1]))/len(labels_collapsed_b)
#0.772

0.7724288840262582

In [ ]:
bag = make_pipeline(StandardScaler(), BaggingClassifier(base_estimator=SVC(gamma='auto', C=6.5), n_estimators=20, random_state=0))
bag.fit(extracted_features_collapsed_e, labels_collapsed)
np.sum(labels_collapsed == bag.predict(extracted_features_collapsed_e))/len(labels_collapsed)

0.8662963697679031

In [ ]:
extracted_features_collapsed_e.shape

(5041, 83)

In [ ]:
gbt = GradientBoostingClassifier(n_estimators=700, learning_rate=0.1, random_state=1)
gbt.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4600])
np.sum(labels_collapsed_b == gbt.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.6340956340956341

In [ ]:
np.sum(labels_collapsed_a == gbt.predict(extracted_features_collapsed_a))/len(labels_collapsed_a)

0.9923913043478261

In [ ]:
np.sum(labels_collapsed_b == gbt.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.6340956340956341

--------------------------------------------------Making predictions-----------------------------------------------

In [58]:
X_test = test_X_tensor.numpy()

In [59]:
X_test.shape

(3411, 17842)

In [60]:
extracted_rpeaks_test=[]

In [61]:
for i in range(len(X_test)):
    signal = X_test[i]
    signal = signal[~np.isnan(signal)] #drop nans
    if (i%500==0):
      print(i)
    res = ecg.ecg(signal, sampling_rate=300, show=False)
    r_peaks = res['rpeaks']
    heart_rates = res["heart_rate"]
    if len(r_peaks) < 5:
      print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # else:
    #   print(f"len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
    extracted_rpeaks_test.append([signal,r_peaks,heart_rates])    

0
500
1000
1500
2000
2500
3000


In [62]:
extracted_features_test = np.ones((len(X_test),5*27+5*4+2+2+2+2*4+1+1+61+10+45))

In [ ]:
for i in range(len(extracted_rpeaks_test)):
    signal, r_peaks, heart_rates = (extracted_rpeaks_test[i][0],extracted_rpeaks_test[i][1],extracted_rpeaks_test[i][2])
    if (i%500==0):
      print(f"processing point {i}")
    if len(r_peaks) >= 5:
        beats = ecg.extract_heartbeats(signal,r_peaks,300,before=0.45,after=0.45)['templates']
        if len(beats) != 0:
            r_diffs = np.diff(r_peaks)
            mu = np.mean(beats, axis=0)[0::10] 
            mini = np.amin(beats, axis=0)[0::10] 
            maxa = np.amax(beats, axis=0)[0::10]  
            var = np.std(beats, axis=0)[0::10] 
            med = np.median(beats, axis=0)[0::10]
            r_mu = np.mean(r_diffs)
            r_mini = np.amin(r_diffs) 
            r_maxa = np.amax(r_diffs)  
            r_var = np.std(r_diffs)
            r_med = np.median(r_diffs)
            r_kurtosis= scipy.stats.kurtosis(r_diffs)
            r_skew = scipy.stats.skew(r_diffs)
            r_20 = np.percentile(r_diffs,20)
            r_40 = np.percentile(r_diffs,40)
            r_60 = np.percentile(r_diffs,60)
            r_80 = np.percentile(r_diffs,80)
            r_hist = np.histogram(r_diffs,bins=14)[1]
            smentropy = ant.sample_entropy(mu)
            appentropy = ant.app_entropy(mu)

            r_diffs_diffs = np.diff(r_diffs)
            r_diffs_mu = np.mean(r_diffs_diffs)
            r_diffs_maxa = np.amax(r_diffs_diffs)
            r_diffs_mini = np.amin(r_diffs_diffs)
            r_diffs_var = np.std(r_diffs_diffs)
            r_diffs_med = np.median(r_diffs_diffs)
            r_diffs_kurtosis= scipy.stats.kurtosis(r_diffs_diffs)
            r_diffs_skew = scipy.stats.skew(r_diffs_diffs)
            r_diffs_25 = np.percentile(r_diffs_diffs,25)
            r_diffs_75 = np.percentile(r_diffs_diffs,75)
            nn_50 = len(r_diffs_diffs[r_diffs_diffs>15]) / len(r_diffs_diffs)
            nn_20 = len(r_diffs_diffs[r_diffs_diffs>6]) / len(r_diffs_diffs)
            c = np.mean(sorted(heart_rates[:-2]))*len(signal)/18000
            rat = np.abs((len(r_peaks)/c)-1)
            cv_rr = r_var/r_mu
            cv_drr = r_diffs_var/r_mu
            r_diffs_hist = np.histogram(r_diffs_diffs,bins=14)[1]

            r_diffs_diffs_2 = np.diff(r_diffs_diffs)
            r_diffs2_mu = np.mean(r_diffs_diffs_2)
            r_diffs2_maxa = np.amax(r_diffs_diffs_2)
            r_diffs2_mini = np.amin(r_diffs_diffs_2)
            r_diffs2_var = np.std(r_diffs_diffs_2)
            r_diffs2_med = np.median(r_diffs_diffs_2)
            r_diffs2 = np.array([r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med])
            r_diffs_2_hist = np.histogram(r_diffs_diffs_2,bins=14)[1]

            abla, lospowers = scipy.signal.welch(signal, 300, 'flattop', 1024, scaling='spectrum')
            #heart rate stats
            if len(heart_rates)>0:
              hr_mu = np.mean(heart_rates)
              hr_maxa = np.amax(heart_rates)
              hr_mini = np.amin(heart_rates)
              hr_var = np.std(heart_rates)
              hr_med = np.median(heart_rates)
            else:
              hr_mu = np.nan
              hr_maxa = np.nan
              hr_mini = np.nan
              hr_var = np.nan
              hr_med = np.nan
            r_stats = np.array([r_mu,r_var,r_mini,r_maxa,r_med,r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med, r_diffs_25, r_diffs_75,hr_mu,hr_maxa,hr_mini,hr_var,hr_med,nn_50,nn_20,smentropy,appentropy,rat,cv_rr,cv_drr,r_skew,r_kurtosis,r_diffs_skew,r_diffs_kurtosis,r_20,r_40,r_60,r_80])
            r_stats = np.concatenate((r_stats,r_diffs2,lospowers[0:10],lospowers[10::10],r_hist,r_diffs_hist,r_diffs_2_hist))

            #HRV STATS

            r_peaks_s = r_peaks/300
            a = welch_psd(rpeaks=r_peaks_s, fbands = fbands, show=False, show_param=False, legend=False)#, fbands=None)
            hrv_stats = np.concatenate((a["fft_peak"],a["fft_abs"],np.array([a["fft_ratio"]])))
            extracted_features_test[i] = np.array(np.concatenate((mu,var,mini,maxa,med,r_stats,hrv_stats)))
    else:  
        print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
        extracted_features_test[i]=np.ones(5*27+5*4+2+2+2+2*4+1+1+61+10+45) #np.array([np.ones(18),np.ones(18),np.ones(18),np.ones(18)])

In [ ]:
# extracted_features_test = np.apply_along_axis(extract_features, 1, X_test)

In [66]:
bool_extracted_test = (extracted_features_test == np.array(np.ones(5*27+5*4+2+2+2+2*4+1+1+61+10+45))).all(axis=1)

In [67]:
extracted_features_test.shape

(3411, 287)

In [68]:
extracted_features_test_valid = extracted_features_test[~bool_extracted_test]
extracted_features_test_valid.shape

(3411, 287)

In [71]:
predictiontest = np.ones(3411)

In [ ]:
#predictiontest[~bool_extracted_test] = xgb.predict(extracted_features_test_valid)

In [ ]:
p1 = xgb.predict(extracted_features_test_valid)

In [69]:
p1 = lgb.predict(extracted_features_test_valid)

In [ ]:
#@title ablaoubla
# from scipy import stats
# comb = np.concatenate((p1[:,np.newaxis],p2[:,np.newaxis],p3[:,np.newaxis]),axis=1)
# comb.shape

(3356, 3)

In [ ]:
# def get_mode(a):
#   return stats.mode(a)[0][0]
p1.shape

(3411,)

In [72]:
predictiontest[~bool_extracted_test]  = p1 #np.apply_along_axis(get_mode, 1, comb)

In [ ]:
predictiontest[bool_extracted_test] = 0

In [ ]:
list(predictiontest).count(3)

121

In [73]:
test_y = pd.read_csv('sample.csv')
test_y.loc[:,"y"] = predictiontest
test_y.to_csv("predictions21.csv", index=False)